<a href="https://colab.research.google.com/github/kato1329/CATech/blob/main/commonlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 7.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder_path = './drive/MyDrive/commonlitcomp/'

In [4]:
summaries_train_path = folder_path + 'summaries_train.csv'
summaries_test_path = folder_path + 'summaries_test.csv'

summaries_train


学生ライターのIDやテキスト、そこから得られたスコアが示されている。カラムの説明は以下の通り


・student_id 学生ライターのID


・prompt_id プロンプトのID


・text テキスト本体


・content 概要のコンテンツスコア（最初のターゲット）


・wording  要約の文言スコア(2番目のターゲット)

In [5]:
prompts_train_path = folder_path + 'prompts_train.csv'
prompts_test_path = folder_path + 'prompts_test.csv'

prompts_train


学生ライターに課された課題、プロンプトについて詳しく述べられている。カラムは以下の通り


・prompt_id

概要ファイルにリンクするプロンプト自体のID

・prompt_question

学生に回答を求めた質問の内容

・prompt_title

プロンプトの短縮したタイトル

・prompt_text

プロンプトの全文

In [7]:
df_train_sum = pd.read_csv(summaries_train_path)
df_test_sum = pd.read_csv(summaries_test_path)

In [8]:
df_train_data = df_train_sum[['student_id','prompt_id','text']]
df_train_target = df_train_sum[['content','wording']]

In [9]:
df_train_target

,content,wording
0,0.205683,0.380538
1,-0.548304,0.506755
2,3.128928,4.231226
3,-0.210614,-0.471415
4,3.272894,3.219757
...,...,...
7160,0.205683,0.380538
7161,-0.308448,0.048171
7162,-1.408180,-0.493603
7163,-0.393310,0.627128


In [10]:
df_train_data

,student_id,prompt_id,text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci..."
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...
...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...


In [11]:
df_train_data.isnull().sum()

student_id    0
prompt_id     0
text          0
dtype: int64

nullデータはない

prompt_idは大きく4つに分けられる。814d6bのプロンプトは一つだけ数が少ないがそれ以外は同じ程度の数を持っている。

データの数は7164個ある。

In [12]:
df_train_target

,content,wording
0,0.205683,0.380538
1,-0.548304,0.506755
2,3.128928,4.231226
3,-0.210614,-0.471415
4,3.272894,3.219757
...,...,...
7160,0.205683,0.380538
7161,-0.308448,0.048171
7162,-1.408180,-0.493603
7163,-0.393310,0.627128


In [13]:
df_train_target.isnull().sum()

content    0
wording    0
dtype: int64

やっぱりデータにnullはなかった

In [14]:
pd.DataFrame(df_train_data[['prompt_id']].value_counts())

,0
prompt_id,
39c16e,2057
3b9047,2009
ebad26,1996
814d6b,1103


prompt_idの種類は四つ以下の通り

・39c16e

・3b9047

・ebad26

・814d6b

In [15]:
df_prompts_train = pd.read_csv(prompts_train_path)
df_prompts_test = pd.read_csv(prompts_test_path)

In [16]:
df_prompts_train.columns

Index(['prompt_id', 'prompt_question', 'prompt_title', 'prompt_text'], dtype='object')

In [17]:
df_prompts_train[['prompt_id','prompt_text']]

,prompt_id,prompt_text
0,39c16e,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,Egyptian society was structured like a pyramid...
2,814d6b,Background \r\nThe Third Wave experiment took ...
3,ebad26,"With one member trimming beef in a cannery, an..."


In [18]:
id_list = ['39c16e','3b9047','ebad26','814d6b']
prompt_dic = {}
question_dic = {}
for id in id_list:
  prompt_dic[id] = df_prompts_train.loc[df_prompts_train['prompt_id'] == id,'prompt_text']
  question_dic[id] = df_prompts_train.loc[df_prompts_train['prompt_id'] == id,'prompt_question']

df_train_data['prompt_text'] = pd.DataFrame(np.zeros(len(df_train_data)))
df_train_data['prompt_question'] = pd.DataFrame(np.zeros(len(df_train_data)))
for id in id_list:
  df_train_data.loc[df_train_data['prompt_id'] == id,'prompt_text'] = str(prompt_dic[id])
  df_train_data.loc[df_train_data['prompt_id'] == id,'prompt_question'] = str(prompt_dic[id])

<ipython-input-18-7d03ae7cfbf9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_data['prompt_text'] = pd.DataFrame(np.zeros(len(df_train_data)))


In [19]:
df_train_data = df_train_data.drop(columns = ['student_id','prompt_id'])

In [20]:
df_train_data.head(1)

,text,prompt_text,prompt_question
0,The third wave was an experimentto see how peo...,2 Background \r\nThe Third Wave experiment ...,2 Background \r\nThe Third Wave experiment ...


In [21]:
df_train_target.head(1)

,content,wording
0,0.205683,0.380538


In [22]:
df_X = pd.concat([df_train_data,df_train_target],axis = 1)

In [23]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(df_X,test_size=0.2)

In [24]:
from transformers import RobertaTokenizer,RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


雑にデータの前処理が終わったのでデータセットにしていく。

In [25]:
from torch.utils.data import Dataset,DataLoader
MAX_LEN = 256
class MyDataset(Dataset):
  def __init__(self,data,tokenizer):
    self.data = pd.DataFrame(data)
    self.tokenizer = tokenizer
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    text_ans = self.data['text'].tolist()[idx]
    text_ques = self.data['prompt_text'].tolist()[idx]
    content = self.data['content'].tolist()[idx]
    wording = self.data['wording'].tolist()[idx]
    content = torch.tensor(content,dtype=torch.float32)
    wording = torch.tensor(wording,dtype=torch.float32)

    ###encoding ans
    encodings_ans = self.tokenizer(
        text_ans,padding='max_length',
        max_length = MAX_LEN,
        truncation = True
    )

    input_ids_ans = torch.tensor(encodings_ans['input_ids'],dtype=torch.long)
    attention_mask_ans = torch.tensor(encodings_ans['attention_mask'],dtype=torch.long)
    #token_type_ids_ans = torch.tensor(encodings_ans['token_type_ids'],dtype=torch.long)

    ###encoding questions
    encodings_questions = self.tokenizer(
        text_ques,padding='max_length',
        max_length=MAX_LEN,
        truncation = True
    )

    input_ids_ques = torch.tensor(encodings_questions['input_ids'],dtype=torch.long)
    attention_mask_ques = torch.tensor(encodings_questions['attention_mask'],dtype=torch.long)
    #token_type_ids_ques = torch.tensor(encodings_questions['token_type_ids'],dtype=torch.long)

    return (input_ids_ans,attention_mask_ans),(input_ids_ques,attention_mask_ques),(content,wording)

In [26]:
train_data = MyDataset(train_data,tokenizer)
test_data = MyDataset(test_data,tokenizer)

In [27]:
train_loader = DataLoader(train_data,batch_size=16,shuffle=True,drop_last=True)
test_loader = DataLoader(test_data,batch_size=16,shuffle=True,drop_last=True)

In [28]:
tmp = iter(train_loader)

for batch in tmp:
  (input_ids_ans,attention_mask_ans),(input_ids_ques,attention_mask_ques),(content,wording) = batch[0],batch[1],batch[2]
  print('input_ids_ansのshapeは',batch[0][0].shape)
  print('attention_mask_ansのshapeは',attention_mask_ans.shape)
  print('-----------------------------------------------------------')
  print('input_ids_quesのshapeは',input_ids_ques.shape)
  print('attention_mask_quesのshapeは',attention_mask_ques.shape)
  print('-----------------------------------------------------------')
  print('contentのshapeは',content.reshape(16,1).shape)
  print('wordingのshapeは',wording.reshape(16,1).shape)
  break


input_ids_ansのshapeは torch.Size([16, 256])
attention_mask_ansのshapeは torch.Size([16, 256])
-----------------------------------------------------------
input_ids_quesのshapeは torch.Size([16, 256])
attention_mask_quesのshapeは torch.Size([16, 256])
-----------------------------------------------------------
contentのshapeは torch.Size([16, 1])
wordingのshapeは torch.Size([16, 1])


In [29]:
sample_text = "Replace me by any text you'd like."
encoded_input = tokenizer(sample_text,return_tensors='pt')
out = model(**encoded_input)
print(out.pooler_output.shape)

torch.Size([1, 768])


In [30]:
class RegRoBERTa(nn.Module):
  def __init__(self):
    super().__init__()
    self.roberta1 = RobertaModel.from_pretrained('roberta-base')
    self.roberta2 = RobertaModel.from_pretrained('roberta-base')
    self.linear1 = nn.Linear(in_features=768*2,out_features=256)
    self.linear2 = nn.Linear(256,64)
    self.linear3 = nn.Linear(64,2)
    self.relu = nn.ReLU()
  def forward(self,
              input_ids_ans,attention_mask_ans,
              input_ids_ques,attention_mask_ques):
    x_ans = self.roberta1(input_ids=input_ids_ans,
                          attention_mask=attention_mask_ans).pooler_output
    x_ques = self.roberta2(input_ids=input_ids_ques,
                            attention_mask=attention_mask_ques).pooler_output
    x = torch.cat((x_ans,x_ques),dim = 1)

    x = self.relu(self.linear1(x))
    x = self.relu(self.linear2(x))
    output = self.linear3(x)

    return output

In [ ]:
print(RegRoBERTa())

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
Model = RegRoBERTa().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(Model.parameters(),lr=1e-5)
epoch = 10

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from tqdm import tqdm

In [38]:
for i in range(epoch):
  print('---------------------')
  print("Epoch:{}/{}".format(i+1,epoch))

  train_loss = 0
  train_accuracy = 0
  test_loss = 0
  test_accuracy = 0

  model.train()

  for batch in tqdm(train_loader):
    optimizer.zero_grad()

    input_ids_ans = batch[0][0].to(device)
    attention_mask_ans = batch[0][1].to(device)

    input_ids_ques = batch[1][0].to(device)
    attention_mask_ques = batch[1][1].to(device)

    target = batch[2]
    content = target[0].reshape(16,1)
    wording = target[1].reshape(16,1)
    target = torch.cat((content,wording),dim=1).to(device)
    pred = Model(input_ids_ans,attention_mask_ans,input_ids_ques,attention_mask_ques)
    loss = criterion(pred,target)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  train_loss = train_loss/len(train_loader)
  print("train_loss = {}".format(train_loss))
  model.eval()
  with torch.no_grad():
    for batch in tqdm(test_loader):
      input_ids_ans = batch[0][0].to(device)
      attention_mask_ans = batch[0][1].to(device)

      input_ids_ques = batch[1][0].to(device)
      attention_mask_ques = batch[1][1].to(device)
      target = batch[2]
      content = target[0].reshape(16,1)
      wording = target[1].reshape(16,1)
      target = torch.cat((content,wording),dim=1).to(device)
      pred = Model(input_ids_ans,attention_mask_ans,input_ids_ques,attention_mask_ques)
      loss = criterion(pred,target)
      test_loss += loss.item()
  test_loss = test_loss/len(test_loader)
  print("test_loss = {}".format(test_loss))

---------------------
Epoch:1/10


100%|██████████| 358/358 [08:02<00:00,  1.35s/it]


train_loss = 0.17732334553196444


100%|██████████| 89/89 [00:40<00:00,  2.20it/s]


test_loss = 0.23465503199716634
---------------------
Epoch:2/10


...

test_loss = 0.2390645712279202
---------------------
Epoch:10/10


100%|██████████| 358/358 [08:04<00:00,  1.35s/it]


train_loss = 0.012128699503256289


100%|██████████| 89/89 [00:41<00:00,  2.16it/s]

test_loss = 0.23904395429940706


In [39]:
torch.save(Model.to('cpu').state_dict(),folder_path+'RegRoBERTa.pth')

In [45]:
df_test_data = df_test_sum
df_test_data['prompt_text'] = pd.DataFrame(np.zeros(len(df_test_data)))
df_test_data['prompt_question'] = pd.DataFrame(np.zeros(len(df_test_data)))

In [47]:
len(df_test_data)

4